In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os

In [21]:
# noinspection PyTypeChecker
class ConvolutionalNeuralNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  # 3: colors - R G B, 6: output layer size, 5: convolution kernel size
        self.pool = nn.MaxPool2d(4, 4)  #4: Pool size, 4: Stride size
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.LazyLinear(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):  # n: number samples in a batch.
        # Start with n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Use local GPU for CNN models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
# Set ML configuration
data_dir = r'../Pneumonia_classification_data/reshape_500'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5
batch_size = 4
rate_learning = 0.001
classes = ('Normal', 'Pneumonia')

In [23]:
# Load and transform datasets
# Images are processed from main.py to 300x300 greyscale jpeg format
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')


transform = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1),
     transforms.ToTensor()])
dataset_test = datasets.ImageFolder(test_dir, transform=transform)
dataset_train = datasets.ImageFolder(train_dir, transform=transform)
dataset_val = datasets.ImageFolder(val_dir, transform=transform)

loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [24]:
# Setup model, loss, optimizer and total training steps
model = ConvolutionalNeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=rate_learning)
n_total_steps = len(loader_train)

C:\Users\Wenda\anaconda3\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [25]:
# Start model training
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print('Finished Training')

Epoch [1/5], Step [100/1243], Loss: 0.6544
Epoch [1/5], Step [200/1243], Loss: 0.6982
Epoch [1/5], Step [300/1243], Loss: 0.5573
Epoch [1/5], Step [400/1243], Loss: 0.6210
Epoch [1/5], Step [500/1243], Loss: 0.6129
Epoch [1/5], Step [600/1243], Loss: 0.6004
Epoch [1/5], Step [700/1243], Loss: 0.5892
Epoch [1/5], Step [800/1243], Loss: 0.5672
Epoch [1/5], Step [900/1243], Loss: 1.1954
Epoch [1/5], Step [1000/1243], Loss: 0.5676
Epoch [1/5], Step [1100/1243], Loss: 0.3105
Epoch [1/5], Step [1200/1243], Loss: 0.5373
Epoch [2/5], Step [100/1243], Loss: 0.3219
Epoch [2/5], Step [200/1243], Loss: 0.5547
Epoch [2/5], Step [300/1243], Loss: 0.5673
Epoch [2/5], Step [400/1243], Loss: 0.3067
Epoch [2/5], Step [500/1243], Loss: 0.8130
Epoch [2/5], Step [600/1243], Loss: 1.1610
Epoch [2/5], Step [700/1243], Loss: 0.5365
Epoch [2/5], Step [800/1243], Loss: 0.5647
Epoch [2/5], Step [900/1243], Loss: 0.3147
Epoch [2/5], Step [1000/1243], Loss: 0.7458
Epoch [2/5], Step [1100/1243], Loss: 0.7854
Epoch 

In [26]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in loader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predictions == labels).sum().item()

        for i in range(len(labels)):
            label = labels[i]
            pred = predictions[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(2):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc:.2f} %')

Accuracy of the network: 73.92739273927393 %
Accuracy of Normal: 35.19 %
Accuracy of Pneumonia: 98.12 %
